<a href="https://colab.research.google.com/github/helen-mark/phonecall/blob/master/MCP_Call_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install zstd
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama

!pip list | grep ollama

!ollama list

import subprocess, time
print("Запускаю Ollama сервер...")
subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(10)

import requests
try:
    response = requests.get("http://localhost:11434/api/tags", timeout=5)
    if response.status_code == 200:
        print("✅ Ollama сервер работает")
    else:
        print(f"⚠️  Ollama сервер отвечает с кодом {response.status_code}")
except:
    print("❌ Ollama сервер не отвечает")

print("Скачиваю модель mistral-nemo:12b...")
!ollama pull mistral-nemo:12b
print('✅ Done')



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (546 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package zstd.
(Reading database ... 121689 files and directories currently i

In [1]:
CONFIG = {
    # GitHub
    "github_repo": "https://github.com/helen-mark/phonecall.git",
    "repo_name": 'phonecall',

    # Google Drive пути
    "drive_root": "/content/drive/MyDrive",
    "project_folder": "MCP_Call_Analytics",

    # Папки для данных
    "folders": {
        "audio_raw": "audio_raw",
        "audio_processed": "audio_processed",
        "csv_calls": "csv_calls",
        "saved_results": "saved_results",
        "logs": "logs"
    },

    # Модели
    "llm_model": "mistral-nemo:12b",

    # Настройки обработки
    "check_interval": 30,  # секунды между проверками новых файлов
    "auto_process": True   # Автоматически обрабатывать новые файлы
}

In [2]:
import os
# ====================
# 🚀 ЗАГРУЗКА КОДА ИЗ GITHUB РЕПОЗИТОРИЯ
# ====================

from google.colab import files
import sys
from google.colab import drive

print("📦 Загружаю проект из GitHub...")

repo_url = CONFIG["github_repo"]
repo_name = CONFIG["repo_name"]

!rm -rf {repo_name} 2>/dev/null || true

!git clone {repo_url}

%cd {repo_name+ '/colab'}

print("\n📁 Project structure:")
!find . -name "*.py" | head -20

project_path = os.getcwd()
sys.path.insert(0, project_path)
print(f"\n📍 Проект загружен в: {project_path}")
print(f"📦 Python path обновлен")

print("\n2️⃣ Монтирую Google Drive...")
drive.mount('/content/drive', force_remount=True)

base_path = os.path.join(CONFIG['drive_root'], CONFIG['project_folder'])

for folder_name, folder_path in CONFIG['folders'].items():
    full_path = os.path.join(base_path, folder_path)
    os.makedirs(full_path, exist_ok=True)
    print(f"   ✅ {folder_name}: {full_path}")

required_files = ["main.py", "interactive.py", "mcp_orchestrator.py"]
missing_files = []

for file in required_files:
    if os.path.exists(file):
        print(f"✅ Найден: {file}")
    else:
        missing_files.append(file)
        print(f"❌ Отсутствует: {file}")

if missing_files:
    print(f"\n⚠️  Отсутствуют файлы: {missing_files}")
    print("Проверьте структуру репозитория на GitHub")
else:
    print("\n🎉 Все файлы загружены успешно!")

# print("📤 ЗАГРУЗИТЕ CSV ФАЙЛЫ ЗВОНКОВ:")
# csv_files = files.upload()

# for filename in csv_files.keys():
#     if filename.endswith('.csv'):
#         os.rename(filename, f"/content/csv_calls/{filename}")
#         print(f"✅ CSV файл: {filename}")



📦 Загружаю проект из GitHub...
Cloning into 'phonecall'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 109 (delta 46), reused 82 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 315.42 KiB | 1.85 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/phonecall/colab

📁 Project structure:
./mcp_orchestrator.py
./colab_minimal.py
./main.py
./colab_drive_setup.py
./interactive.py

📍 Проект загружен в: /content/phonecall/colab
📦 Python path обновлен

2️⃣ Монтирую Google Drive...
Mounted at /content/drive
   ✅ audio_raw: /content/drive/MyDrive/MCP_Call_Analytics/audio_raw
   ✅ audio_processed: /content/drive/MyDrive/MCP_Call_Analytics/audio_processed
   ✅ csv_calls: /content/drive/MyDrive/MCP_Call_Analytics/csv_calls
   ✅ saved_results: /content/drive/MyDrive/MCP_Call_Analytics/saved_results
   ✅ logs: /content/drive/MyDrive/MCP_Call_Analytics/logs
✅ Найден: m

In [4]:
!pip install pydub

In [3]:
print("📦 Устанавливаю OpenAI Whisper...")
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 2. Проверьте установку
import whisper
print(f"✅ Whisper установлен, версия: {whisper.__version__}")

# 3. Скачайте модель (выберите нужный размер)
MODEL_SIZE = "large"  # или "small", "medium", "large", "large-v3"
# print(f"📥 Скачиваю модель {MODEL_SIZE}...")
# model = whisper.load_model(MODEL_SIZE)
# print("✅ Модель загружена")

📦 Устанавливаю OpenAI Whisper...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ Whisper установлен, версия: 20250625


In [4]:
import json
import pandas as pd

"""Явное указание пути"""
import sys

# Явно добавляем путь к вашему репозиторию
phonecall_path = "/content/phonecall"  # ИЗМЕНИТЕ НА ВАШ ПУТЬ
if phonecall_path not in sys.path:
    sys.path.insert(0, phonecall_path)

from phonecall.preprocess_calls_full import AudioProcessor


class AudioFileMonitor:
    """Мониторит новые аудиофайлы в папке audio_raw"""

    def __init__(self, audio_raw_path, csv_path, log_path):
        self.audio_raw_path = audio_raw_path
        self.csv_path = csv_path
        self.log_path = log_path
        self.processed_files = self._load_processed_files()

        os.makedirs(log_path, exist_ok=True)
        self.log_file = os.path.join(log_path, "audio_processing.log")

    def _load_processed_files(self):
        """Загружает список уже обработанных файлов из CSV"""
        processed = set()

        # Проверяем CSV файлы
        if os.path.exists(self.csv_path):
            csv_files = [f for f in os.listdir(self.csv_path) if f.endswith('.csv')]
            for csv_file in csv_files:
                try:
                    df = pd.read_csv(os.path.join(self.csv_path, csv_file))
                    if 'source_audio' in df.columns:
                        processed.update(df['source_audio'].dropna().unique())
                except:
                    pass

        # # Также проверяем логи
        # log_file = os.path.join(self.log_path, "processed_audio.json")
        # if os.path.exists(log_file):
        #     with open(log_file, 'r') as f:
        #         processed.update(json.load(f))

        return processed

    def check_new_files(self):
        """Проверяет наличие новых аудиофайлов"""
        if not os.path.exists(self.audio_raw_path):
            return []

        # Получаем все аудиофайлы
        audio_extensions = ['.mp3', '.wav', '.m4a', '.flac', '.ogg']
        all_files = []

        for ext in audio_extensions:
            all_files.extend(Path(self.audio_raw_path).glob(f"*{ext}"))
            all_files.extend(Path(self.audio_raw_path).glob(f"*{ext.upper()}"))

        # Фильтруем новые файлы
        new_files = []
        for file_path in all_files:
            file_name = file_path.name
            if file_name not in self.processed_files:
                new_files.append(str(file_path))

        return new_files

    def log_processing(self, audio_file, status, details=""):
        """Логирует процесс обработки"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"{timestamp} | {audio_file} | {status} | {details}\n"

        # Записываем в лог-файл
        with open(self.log_file, 'a', encoding='utf-8') as f:
            f.write(log_entry)

        # Обновляем список обработанных файлов
        if status == "COMPLETED":
            self.processed_files.add(os.path.basename(audio_file))
            self._save_processed_files()

        print(f"📝 {log_entry.strip()}")

    def _save_processed_files(self):
        """Сохраняет список обработанных файлов"""
        log_file = os.path.join(self.log_path, "processed_audio.json")
        with open(log_file, 'w') as f:
            json.dump(list(self.processed_files), f)

# Инициализируем монитор
AUDIO_RAW_PATH = os.path.join(base_path, CONFIG['folders']['audio_raw'])
CSV_PATH = os.path.join(base_path, CONFIG['folders']['csv_calls'])
LOG_PATH = os.path.join(base_path, CONFIG['folders']['logs'])

#monitor = AudioFileMonitor(AUDIO_RAW_PATH, CSV_PATH, LOG_PATH)

In [5]:
import os
import json
import time
import shutil
import pandas as pd
from datetime import datetime
from pathlib import Path
import subprocess
from phonecall.preprocess_calls_full import AudioProcessor
from phonecall.assign_tags_from_fixed_list import JsonFileTaggingAgent


class SmartAudioProcessor:
    """Полная реализация инкрементальной обработки с батчами"""

    def __init__(self, drive_audio_path, output_csv_path,
                 total_space_gb=80, batch_size_gb=2):
        """
        Args:
            drive_audio_path: путь к аудиофайлам в Google Drive
            output_csv_path: путь для сохранения CSV результатов
            total_space_gb: общее место в Colab (по умолчанию 80GB)
            batch_size_gb: размер одного батча для обработки (по умолчанию 2GB)
        """
        my_tags = [
        "низкое_качество_стирки_или_чистки",
        "не_заменили_ковры_вовремя",
        "клиент_хочет_добавить_ковры",
        "клиент_хочет_меньше_ковров",
        "погашение_долга",
        "расторжение_договора",
        "возобновление_услуг",
        "долго_нет_ответа_на_заявку",
        "лишняя_доставка",
        "доставили_не_те_ковры",
        "не_выставлен_вовремя_счет",
        "неверная_сумма_в_счете",
        "ковер_забрали_без_причины",
        "забрали_не_тот_ковер",
        "менеджер_нагрубил_клиенту",
        "неоправданно_высокие_цены",
        "неоправданный_рост_цен",
        "новый_клиент_заключение_договора",
        "консультация_или_уточнение_деталей",
        "поменять_спецификации",
        "менеджер_обещал_но_не_связался_с_клиентом",
        "клиент_уходит_к_конкурентам",
        "приостановить_услуги",
        "ошибка_в_документах"
          ]
        self.drive_audio_path = drive_audio_path
        self.output_csv_path = output_csv_path
        self.total_space = total_space_gb
        self.batch_size = batch_size_gb
        self.ap = AudioProcessor(model_size='large')
        self.tagger = JsonFileTaggingAgent(
          model_name="mistral-nemo:12b",
          tags_list=my_tags
        )

        # Локальные рабочие директории
        self.local_temp_dir = "/content/temp_audio"
        self.local_whisper_dir = "/content/whisper_output"
        self.local_batch_dir = "/content/current_batch"

        # Создаем локальные директории
        for dir_path in [self.local_temp_dir, self.local_whisper_dir, self.local_batch_dir]:
            os.makedirs(dir_path, exist_ok=True)

        # Файл для логирования обработанных файлов
        self.processed_files_log = "/content/drive/MyDrive/MCP_Call_Analytics/processed_files.json"

        print(f"🎯 Инициализация SmartAudioProcessor")
        print(f"📁 Аудиофайлы: {drive_audio_path}")
        print(f"💾 Лимит батча: {batch_size_gb} GB")
        print(f"📊 Общее место: {total_space_gb} GB")

    # ========== ОСНОВНОЙ МЕТОД ==========

    def process_large_dataset(self):
        """Основной метод: обрабатывает весь датасет батчами"""

        print("="*60)
        print("🚀 НАЧАЛО ОБРАБОТКИ БОЛЬШОГО ДАТАСЕТА")
        print("="*60)

        # 1. Получаем ВСЕ файлы (только имена, не скачивая)
        all_files = self._list_files_without_download()
        if not all_files:
            print("❌ Нет файлов для обработки")
            return []

        total_files = len(all_files)
        total_gb = self._estimate_total_size_gb(all_files)

        print(f"📊 Всего файлов: {total_files:,}")
        print(f"📊 Общий объем: {total_gb:.1f} GB")
        print(f"🎯 Размер батча: {self.batch_size} GB")
        print(f"🎯 Примерное количество батчей: {int(total_gb / self.batch_size) + 1}")

        # 2. Определяем, какие уже обработаны
        processed = self._load_processed_list()
        to_process = [f for f in all_files if f not in processed]

        print(f"\n📈 СТАТУС ОБРАБОТКИ:")
        print(f"✅ Уже обработано: {len(processed):,} файлов")
        print(f"🎯 Осталось обработать: {len(to_process):,} файлов")
        print(f"📊 Прогресс: {len(processed)/total_files*100:.1f}%")

        if not to_process:
            print("\n🎉 Все файлы уже обработаны!")
            return []

        # 3. Группируем файлы по батчам
        batches = self._create_batches(to_process)
        print(f"\n📦 Создано {len(batches)} батчей для обработки")

        # 4. Обрабатываем каждый батч
        all_results = []
        start_time = time.time()

        for batch_num, batch_files in enumerate(batches, 1):
            batch_start = time.time()

            print(f"\n{'='*50}")
            print(f"⚡ БАТЧ {batch_num}/{len(batches)}")
            print(f"{'='*50}")

            batch_size_gb = sum(self._get_file_size_gb(f) for f in batch_files)
            print(f"📁 Файлов в батче: {len(batch_files)}")
            print(f"💾 Размер батча: {batch_size_gb:.2f} GB")

            # Обрабатываем батч
            try:
                batch_results = self._process_batch(batch_files, batch_num)

                if batch_results:
                    all_results.extend(batch_results)

                    # Сразу дозаписываем результаты в основной CSV
                    self._append_to_main_csv(batch_results)

                    # Обновляем список обработанных
                    self._update_processed_list(batch_files)

                # Расчет времени
                batch_time = time.time() - batch_start
                avg_time_per_file = batch_time / len(batch_files) if batch_files else 0

                print(f"\n✅ Батч {batch_num} завершен за {batch_time:.1f} сек")
                print(f"⏱️  Среднее время на файл: {avg_time_per_file:.1f} сек")

                # Прогноз оставшегося времени
                remaining_batches = len(batches) - batch_num
                if remaining_batches > 0:
                    estimated_remaining = remaining_batches * (time.time() - start_time) / batch_num
                    print(f"📅 Осталось времени: ~{estimated_remaining/60:.1f} мин")

            except Exception as e:
                print(f"❌ Ошибка обработки батча {batch_num}: {e}")
                import traceback
                traceback.print_exc()

                # Пытаемся продолжить со следующим батчем
                continue

        # 5. Финальная проверка и удаление дубликатов
        self._remove_duplicates_from_csv()

        total_time = time.time() - start_time
        print(f"\n{'='*60}")
        print(f"🎉 ОБРАБОТКА ЗАВЕРШЕНА!")
        print(f"{'='*60}")
        print(f"📊 Обработано файлов: {len(all_results):,}")
        print(f"⏱️  Общее время: {total_time/60:.1f} мин")
        print(f"⚡ Скорость: {len(all_results)/total_time*60:.1f} файлов/мин")
        print(f"💾 Результаты сохранены в: {self.output_csv_path}")

        return all_results

    # ========== ИСПРАВЛЕННЫЕ МЕТОДЫ ==========

    def _append_to_main_csv(self, batch_results):
        """Дозаписывает результаты батча в основной CSV файл"""
        if not batch_results:
            print("⚠️  Нет результатов для сохранения")
            return

        # Создаем DataFrame из результатов батча
        df_batch = pd.DataFrame(batch_results)

        # Если основной файл существует, загружаем его и объединяем
        if os.path.exists(self.output_csv_path):
            try:
                # Читаем существующий файл
                df_existing = pd.read_csv(self.output_csv_path)

                # Объединяем старые и новые данные
                df_combined = pd.concat([df_existing, df_batch], ignore_index=True)

                # Сохраняем обратно
                df_combined.to_csv(self.output_csv_path, index=False, encoding='utf-8')

                print(f"💾 Дозаписано {len(batch_results)} записей в {self.output_csv_path}")
                print(f"📊 Теперь в файле: {len(df_combined)} записей")

            except Exception as e:
                print(f"❌ Ошибка дозаписи в CSV: {e}")
                # Пытаемся сохранить как новый файл
                df_batch.to_csv(self.output_csv_path, index=False, encoding='utf-8')
                print(f"💾 Создан новый файл с {len(batch_results)} записями")
        else:
            # Если файла нет, создаем новый
            df_batch.to_csv(self.output_csv_path, index=False, encoding='utf-8')
            print(f"💾 Создан новый файл {self.output_csv_path} с {len(batch_results)} записями")

    def _remove_duplicates_from_csv(self):
        """Удаляет дубликаты из CSV файла по названию файла"""
        if not os.path.exists(self.output_csv_path):
            return

        try:
            df = pd.read_csv(self.output_csv_path)

            # Проверяем наличие нужной колонки
            if 'source_audio' in df.columns:
                # Подсчитываем дубликаты до удаления
                duplicates = df.duplicated(subset=['source_audio'], keep='last').sum()

                if duplicates > 0:
                    print(f"🔍 Найдено {duplicates} дубликатов, удаляю...")

                    # Удаляем дубликаты, оставляем последнюю запись
                    df_clean = df.drop_duplicates(subset=['source_audio'], keep='last')

                    # Сохраняем очищенный файл
                    df_clean.to_csv(self.output_csv_path, index=False, encoding='utf-8')

                    print(f"🧹 Удалено {duplicates} дубликатов")
                    print(f"📊 Осталось {len(df_clean)} уникальных записей")
            else:
                print("⚠️  Колонка 'source_audio' не найдена в CSV")

        except Exception as e:
            print(f"⚠️  Ошибка при удалении дубликатов: {e}")

    def _load_processed_list(self):
        """Загружает список уже обработанных файлов"""
        processed_files = set()

        # 1. Проверяем основной CSV файл с результатами
        if os.path.exists(self.output_csv_path):
            try:
                df = pd.read_csv(self.output_csv_path)
                if 'source_audio' in df.columns:
                    csv_processed = set(df['source_audio'].dropna().unique())
                    processed_files.update(csv_processed)
                    print(f"📊 Из основного CSV загружено: {len(csv_processed)} файлов")
            except Exception as e:
                print(f"⚠️  Ошибка чтения основного CSV: {e}")

        # 2. Проверяем файл с логами обработанных (для истории)
        # if os.path.exists(self.processed_files_log):
        #     try:
        #         with open(self.processed_files_log, 'r') as f:
        #             data = json.load(f)
        #             if isinstance(data, list):
        #                 log_processed = set(data)
        #                 processed_files.update(log_processed)
        #                 print(f"📖 Из лога загружено: {len(log_processed)} файлов")
        #     except Exception as e:
        #         print(f"⚠️  Ошибка чтения лога: {e}")

        # Преобразуем в полные пути для сравнения
        full_paths_processed = set()
        for filename in processed_files:
            full_path = os.path.join(self.drive_audio_path, filename)
            full_paths_processed.add(full_path)

        return full_paths_processed

    def _update_processed_list(self, processed_files):
        """Обновляет список обработанных файлов"""
        try:
            # Загружаем текущий список из лога
            current_processed = []
            if os.path.exists(self.processed_files_log):
                with open(self.processed_files_log, 'r') as f:
                    current_processed = json.load(f)

            # Добавляем новые файлы (только имена)
            new_processed = [os.path.basename(f) for f in processed_files]
            current_processed.extend(new_processed)

            # Удаляем дубликаты
            current_processed = list(set(current_processed))

            # Сохраняем
            with open(self.processed_files_log, 'w') as f:
                json.dump(current_processed, f)

            print(f"📝 Лог обновлен: +{len(new_processed)} файлов, всего {len(current_processed)}")

        except Exception as e:
            print(f"⚠️  Ошибка обновления лога: {e}")

    def _process_batch(self, batch_files, batch_num):
        """Обрабатывает один батч файлов"""
        batch_results = []

        # 1. Очищаем локальные директории
        self._cleanup_local_dirs()

        # 2. Скачиваем файлы батча в локальную ФС
        local_files = []
        print(f"📥 Скачиваю {len(batch_files)} файлов в локальную ФС...")

        for file_path in batch_files:
            try:
                filename = os.path.basename(file_path)
                local_path = os.path.join(self.local_batch_dir, filename)

                # Копируем из Drive в локальную ФС
                shutil.copy2(file_path, local_path)
                local_files.append(local_path)

                # Проверяем, что файл скопировался
                if os.path.exists(local_path):
                    size_mb = os.path.getsize(local_path) / (1024*1024)
                    print(f"  ✅ {filename} ({size_mb:.1f} MB)")
                else:
                    print(f"  ❌ {filename} - не скопировался")

            except Exception as e:
                print(f"  ❌ Ошибка копирования {os.path.basename(file_path)}: {e}")

        print(f"📊 Скачано: {len(local_files)}/{len(batch_files)} файлов")

        if not local_files:
            print("⚠️  Нет файлов для обработки в этом батче")
            return []

        # 3. Обрабатываем каждый локальный файл
        print("\n🔊 Начинаю обработку аудио...")

        for i, local_file in enumerate(local_files, 1):
            try:
                print(f"\n[{i}/{len(local_files)}] Обрабатываю: {os.path.basename(local_file)}")

                # 3.1 Whisper: Аудио → Текст
                print("  📝 Шаг 1: Конвертация аудио в текст...")
                whisper_result = self._run_whisper_locally(local_file)

                if not whisper_result or 'text' not in whisper_result:
                    print("  ⚠️  Whisper не вернул текст")
                    continue

                text = whisper_result['text']
                print(f"  ✅ Текст извлечен ({len(text)} символов)")

                # 3.2 LLM: Текст → Теги
                print("  🏷️  Шаг 2: Тегирование текста...")
                tagging_result = self._run_tagging_locally(text)

                tags = tagging_result.get('result', [])
                summary = tagging_result.get('summary', '')

                print(f"  ✅ Добавлено тегов: {len(tags)}")

                # 3.3 Создаем результат
                result = {
                    'source_audio': os.path.basename(local_file),
                    'text': text,
                    'tags': str(tags),  # Сохраняем как строку
                    'summary': summary,
                    'text_length': len(text),
                    'processing_date': datetime.now().isoformat(),
                    'batch_number': batch_num,
                    'whisper_model': whisper_result.get('model', 'unknown'),
                    'audio_duration': whisper_result.get('duration', 0)
                }

                # Добавляем дополнительные поля из whisper_result
                if 'date' in whisper_result:
                    result['date'] = whisper_result['date']
                if 'quality_score' in whisper_result:
                    result['quality_score'] = whisper_result['quality_score']

                batch_results.append(result)

                # 3.4 Удаляем обработанный локальный файл
                os.remove(local_file)
                print(f"  🧹 Локальный файл удален")

            except Exception as e:
                print(f"  ❌ Ошибка обработки файла: {e}")
                import traceback
                traceback.print_exc()

        # 4. Очищаем временные файлы
        self._cleanup_local_dirs()

        print(f"\n✅ Батч {batch_num} обработан: {len(batch_results)}/{len(batch_files)} успешно")

        return batch_results

    def _run_whisper_locally(self, local_audio_path):
        """Запускает Whisper для локального аудиофайла"""
        try:
            return self.ap.process_file(local_audio_path, 7, False)
        except Exception as e:
            print(f"❌ Ошибка Whisper: {e}")
            return {'text': '', 'error': str(e)}

    def _run_tagging_locally(self, text):
        """Запускает тегирование для текста"""
        try:
            return self.tagger.get_tags_from_llm(text)
        except Exception as e:
            print(f"❌ Ошибка тегирования: {e}")
            return {'result': [], 'summary': f'Ошибка: {str(e)}'}

    # ========== ВСПОМОГАТЕЛЬНЫЕ МЕТОДЫ (остаются без изменений) ==========

    def _list_files_without_download(self):
        """Получает список всех аудиофайлов без их скачивания"""
        if not os.path.exists(self.drive_audio_path):
            print(f"❌ Папка не найдена: {self.drive_audio_path}")
            return []

        audio_extensions = ['.mp3', '.wav', '.m4a', '.flac', '.ogg', '.aac', '.wma']

        all_files = []
        for ext in audio_extensions:
            pattern = f"*{ext}"
            files = list(Path(self.drive_audio_path).glob(pattern.lower()))
            files.extend(Path(self.drive_audio_path).glob(pattern.upper()))

            for file_path in files:
                if file_path.is_file():
                    all_files.append(str(file_path))

        all_files.sort()

        print(f"🔍 Найдено {len(all_files)} аудиофайлов")
        return all_files

    def _estimate_total_size_gb(self, file_paths):
        """Оценивает общий размер файлов в GB"""
        total_bytes = 0

        for file_path in file_paths:
            try:
                total_bytes += os.path.getsize(file_path)
            except:
                total_bytes += 50 * 1024 * 1024  # 50 MB

        return total_bytes / (1024**3)

    def _get_file_size_gb(self, file_path):
        """Возвращает размер файла в GB"""
        try:
            return os.path.getsize(file_path) / (1024**3)
        except:
            return 0.05  # Предполагаем 50 MB

    def _create_batches(self, file_paths):
        """Создает батчи файлов на основе размера"""
        batches = []
        current_batch = []
        current_batch_size = 0

        for file_path in file_paths:
            file_size_gb = self._get_file_size_gb(file_path)

            if current_batch_size + file_size_gb > self.batch_size and current_batch:
                batches.append(current_batch.copy())
                current_batch = []
                current_batch_size = 0

            current_batch.append(file_path)
            current_batch_size += file_size_gb

        if current_batch:
            batches.append(current_batch)

        optimized_batches = self._optimize_batches(batches)

        return optimized_batches

    def _optimize_batches(self, batches):
        """Оптимизирует распределение файлов по батчам"""
        if len(batches) <= 1:
            return batches

        optimized = []
        current_batch = []
        current_size = 0

        for batch in batches:
            batch_size = sum(self._get_file_size_gb(f) for f in batch)

            if current_size + batch_size <= self.batch_size * 1.2:
                current_batch.extend(batch)
                current_size += batch_size
            else:
                if current_batch:
                    optimized.append(current_batch)
                current_batch = batch.copy()
                current_size = batch_size

        if current_batch:
            optimized.append(current_batch)

        print(f"🔧 Оптимизировано: {len(batches)} → {len(optimized)} батчей")

        return optimized

    def _cleanup_local_dirs(self):
        """Очищает локальные директории"""
        for dir_path in [self.local_temp_dir, self.local_whisper_dir, self.local_batch_dir]:
            if os.path.exists(dir_path):
                for file in os.listdir(dir_path):
                    try:
                        os.remove(os.path.join(dir_path, file))
                    except:
                        pass

        print("🧹 Локальные директории очищены")

    def get_processing_stats(self):
        """Возвращает статистику обработки"""
        stats = {
            'total_files': 0,
            'processed_files': 0,
            'remaining_files': 0,
            'progress_percent': 0,
            'estimated_size_gb': 0,
            'last_processed': None
        }

        try:
            all_files = self._list_files_without_download()
            stats['total_files'] = len(all_files)
            stats['estimated_size_gb'] = self._estimate_total_size_gb(all_files)

            processed = self._load_processed_list()
            stats['processed_files'] = len(processed)
            stats['remaining_files'] = stats['total_files'] - stats['processed_files']

            if stats['total_files'] > 0:
                stats['progress_percent'] = (stats['processed_files'] / stats['total_files']) * 100

            if os.path.exists(self.output_csv_path):
                mod_time = os.path.getmtime(self.output_csv_path)
                stats['last_processed'] = datetime.fromtimestamp(mod_time).strftime("%d.%m.%Y %H:%M")

        except Exception as e:
            print(f"⚠️  Ошибка получения статистики: {e}")

        return stats

# ============================================
# 🎯 ИСПОЛЬЗОВАНИЕ
# ============================================

def main():
    """Пример использования SmartAudioProcessor"""

    # Конфигурация
    DRIVE_AUDIO_PATH = "/content/drive/MyDrive/MCP_Call_Analytics/audio_raw"
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/MCP_Call_Analytics/csv_calls/calls.csv"

    print("🤖 SMART AUDIO PROCESSOR - ЗАПУСК")
    print("="*50)

    # Инициализация процессора
    processor = SmartAudioProcessor(
        drive_audio_path=DRIVE_AUDIO_PATH,
        output_csv_path=OUTPUT_CSV_PATH,
        total_space_gb=50,
        batch_size_gb=2
    )

    # Показываем статистику
    stats = processor.get_processing_stats()
    print(f"\n📊 СТАТИСТИКА ДО ОБРАБОТКИ:")
    print(f"📁 Всего файлов: {stats['total_files']:,}")
    print(f"✅ Обработано: {stats['processed_files']:,}")
    print(f"🎯 Осталось: {stats['remaining_files']:,}")
    print(f"📈 Прогресс: {stats['progress_percent']:.1f}%")
    print(f"💾 Общий объем: {stats['estimated_size_gb']:.1f} GB")

    if stats['last_processed']:
        print(f"📅 Последняя обработка: {stats['last_processed']}")

    if stats['remaining_files'] == 0:
        print("\n🎉 Все файлы уже обработаны!")
        return

    # Запрашиваем подтверждение
    confirm = input(f"\n⚠️  Начать обработку {stats['remaining_files']:,} файлов? (y/n): ")

    if confirm.lower() != 'y':
        print("🚫 Обработка отменена")
        return

    # Запускаем обработку
    print("\n🚀 ЗАПУСК ОБРАБОТКИ...")
    results = processor.process_large_dataset()

    # Финальная статистика
    if results:
        print(f"\n🎉 ОБРАБОТКА ЗАВЕРШЕНА УСПЕШНО!")
        print(f"📊 Обработано записей: {len(results):,}")
        print(f"💾 Результаты: {OUTPUT_CSV_PATH}")

        # Показываем содержимое файла
        if os.path.exists(OUTPUT_CSV_PATH):
            df = pd.read_csv(OUTPUT_CSV_PATH)
            print(f"📄 Размер файла: {len(df)} строк, {len(df.columns)} колонок")
            print("📋 Колонки:", list(df.columns))
    else:
        print("\n⚠️  Обработка завершена без результатов")

if __name__ == "__main__":
    main()

🤖 SMART AUDIO PROCESSOR - ЗАПУСК
Загрузка модели Whisper...
Модель загружена!
🎯 Инициализация SmartAudioProcessor
📁 Аудиофайлы: /content/drive/MyDrive/MCP_Call_Analytics/audio_raw
💾 Лимит батча: 2 GB
📊 Общее место: 50 GB
🔍 Найдено 3 аудиофайлов
📊 Из основного CSV загружено: 196 файлов

📊 СТАТИСТИКА ДО ОБРАБОТКИ:
📁 Всего файлов: 3
✅ Обработано: 196
🎯 Осталось: -193
📈 Прогресс: 6533.3%
💾 Общий объем: 0.0 GB
📅 Последняя обработка: 28.01.2026 08:33

⚠️  Начать обработку -193 файлов? (y/n): y

🚀 ЗАПУСК ОБРАБОТКИ...
🚀 НАЧАЛО ОБРАБОТКИ БОЛЬШОГО ДАТАСЕТА
🔍 Найдено 3 аудиофайлов
📊 Всего файлов: 3
📊 Общий объем: 0.0 GB
🎯 Размер батча: 2 GB
🎯 Примерное количество батчей: 1
📊 Из основного CSV загружено: 196 файлов

📈 СТАТУС ОБРАБОТКИ:
✅ Уже обработано: 196 файлов
🎯 Осталось обработать: 2 файлов
📊 Прогресс: 6533.3%

📦 Создано 1 батчей для обработки

⚡ БАТЧ 1/1
📁 Файлов в батче: 2
💾 Размер батча: 0.00 GB
🧹 Локальные директории очищены
📥 Скачиваю 2 файлов в локальную ФС...
  ✅ 2025-10-31_15-37-31.087

In [5]:
import sys
sys.path.append('.')

from colab.interactive import enhanced_interactive_mode

enhanced_interactive_mode(
    _model="mistral-nemo:12b",
    csv_dir=CSV_PATH,
    results_dir="/content/saved_results",
    drive_path=None
)


╔══════════════════════════════════════════╗
║    🤖 АНАЛИТИК ЗВОНКОВ v3.0               ║
║    🚀 Google Drive Edition                ║
╚══════════════════════════════════════════╝
    
✅ Ollama подключен, модель: mistral-nemo:12b
📂 Загружаю данные из JSON файлов...
📂 Читаю данные из CSV файла: calls.csv
✅ Загружено 202 строк из CSV
✅ Преобразовано 202 записей звонков
📅 Диапазон дат: 01.10.2025 - 20.01.2026
🏷️  Уникальных тегов: 19
✅ Загружено 202 звонков

[Local] 📝 Вопрос (/? для помощи): на что чаще всего жаловались клиенты в октябре 2025 года
🔍 Анализирую: 'на что чаще всего жаловались клиенты в октябре 2025 года'

🔍 Анализирую запрос: 'на что чаще всего жаловались клиенты в октябре 2025 года'
🤖 Создаю план анализа...
   📅 Период: октябрь 2025 года
   🏷️  Теги: низкое_качество_стирки_или_чистки, не_заменили_ковры_вовремя, клиент_хочет_добавить_ковры, клиент_хочет_меньше_ковров, погашение_долга, расторжение_договора, возобновление_услуг, долго_нет_ответа_на_заявку, лишняя_доставка, д